# Structure


## Setup

In [4]:
# !pip install -r requirements.txt
!pip install sentence-transformers
!pip install mteb
!pip install beir
!pip install datasets

  Using cached beir-2.0.0-py3-none-any.whl
  Using cached pytrec_eval-0.5.tar.gz (15 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached faiss_cpu-1.8.0.post1-cp312-cp312-win_amd64.whl.metadata (3.8 kB)
  Using cached elasticsearch-7.9.1-py2.py3-none-any.whl.metadata (8.0 kB)
Using cached elasticsearch-7.9.1-py2.py3-none-any.whl (219 kB)
Using cached faiss_cpu-1.8.0.post1-cp312-cp312-win_amd64.whl (14.6 MB)
  Running setup.py clean for pytrec-eval
Failed to build pytrec-eval


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [26 lines of output]
      Fetching trec_eval from https://github.com/usnistgov/trec_eval/archive/v9.0.8.tar.gz.
      C:\Users\Willi\miniconda3\Lib\site-packages\setuptools\dist.py:476: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              By 2024-Sep-26, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              *********************************************************************

## Matryoshka-Adaptor Implementation

### Architecture

In [2]:
import torch
import torch.nn.functional as F

# Define MatryoshkaAdaptor module - a simple MLP
class MatryoshkaAdaptor(torch.nn.Module):
    """
    A PyTorch neural network module that adapts the output of an embedding model
    into a desired output dimension using two linear transformations with a ReLU activation in between.

    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MatryoshkaAdaptor, self).__init__()
        # First linear layer to transform the input dimension to a hidden dimension
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        # Second linear layer to transform the hidden dimension to the output dimension
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        # Activation function to introduce non-linearity
        self.activation = torch.nn.ReLU()

    def forward(self, embedding):
        # Apply the first linear transformation followed by the activation function
        hidden_embedding = self.activation(self.linear1(embedding))
        
        # Apply the second linear transformation to get the final adapted embedding
        adapted_embedding = self.linear2(hidden_embedding)

        return adapted_embedding

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Willi\miniconda3\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

### Loss Functions

In [3]:
# Equation 1 in paper
def pairwise_similarity_loss(original_embeddings, matryoshka_embeddings):
    """
    Computes the pairwise similarity loss between original embeddings and matryoshka embeddings.
    
    Args:
        original_embeddings: A tensor of shape (batch_size, embedding_dim) representing the original embeddings.
        matryoshka_embeddings: A tensor of shape (batch_size, embedding_dim) representing the matryoshka embeddings.
        
    Returns:
        loss: A scalar tensor representing the mean pairwise similarity loss.
    """
    
    # Normalize the embeddings along the embedding dimension to get the cosine similarity
    normalized_original_embeddings = F.normalize(original_embeddings, p=2, dim=1)
    normalized_matryoshka_embeddings = F.normalize(matryoshka_embeddings, p=2, dim=1)
    
    # Compute the cosine similarity matrices
    original_similarity_matrix = torch.matmul(normalized_original_embeddings, normalized_original_embeddings.T)
    matryoshka_similarity_matrix = torch.matmul(normalized_matryoshka_embeddings, normalized_matryoshka_embeddings.T)
    
    # Get the indices of the upper triangle of the matrices, excluding the diagonal
    batch_size = original_embeddings.size(0)
    i, j = torch.triu_indices(batch_size, batch_size, offset=1)
    
    # Compute the pairwise cosine similarities
    original_pairwise_similarities = original_similarity_matrix[i, j]
    matryoshka_pairwise_similarities = matryoshka_similarity_matrix[i, j]
    
    # Compute the absolute difference between corresponding pairwise similarities
    similarity_differences = torch.abs(original_pairwise_similarities - matryoshka_pairwise_similarities)
    
    # Sum up all the absolute differences to produce the final loss
    loss = torch.sum(similarity_differences)
    
    return loss

# Equation 2 in paper
def topk_similarity_loss(original_embeddings, matryoshka_embeddings, k=5):
    """
    Computes the top-k similarity loss between original embeddings and matryoshka embeddings.
    
    Args:
        original_embeddings: A tensor of shape (batch_size, embedding_dim) representing the original embeddings.
        matryoshka_embeddings: A tensor of shape (batch_size, embedding_dim) representing the matryoshka embeddings.
        k: The number of top similarities to consider (default is 5).
        
    Returns:
        loss: A scalar tensor representing the top-k similarity loss.
    """
    
    # Normalize the embeddings along the embedding dimension to get the cosine similarity
    normalized_original_embeddings = F.normalize(original_embeddings, p=2, dim=1)
    normalized_matryoshka_embeddings = F.normalize(matryoshka_embeddings, p=2, dim=1)
    
    # Compute the cosine similarity matrices
    original_similarity_matrix = torch.matmul(normalized_original_embeddings, normalized_original_embeddings.T)
    matryoshka_similarity_matrix = torch.matmul(normalized_matryoshka_embeddings, normalized_matryoshka_embeddings.T)
    
    # Exclude self-similarity by setting the diagonal to a very low value
    batch_size = original_embeddings.size(0)
    original_similarity_matrix.fill_diagonal_(-float('inf'))
    matryoshka_similarity_matrix.fill_diagonal_(-float('inf'))
    
    # For each embedding, get the top-k similarities and their corresponding indices
    original_topk_values, _ = torch.topk(original_similarity_matrix, k, dim=1)
    matryoshka_topk_values, _ = torch.topk(matryoshka_similarity_matrix, k, dim=1)
    
    # Compute the absolute difference between the top-k similarities
    similarity_differences = torch.abs(original_topk_values - matryoshka_topk_values)
    
    # Sum up all the absolute differences to produce the final loss
    loss = torch.sum(similarity_differences)
    
    return loss


# Equation 3 in paper
def regularization_loss(original_embeddings, matryoshka_embeddings, alpha=1.0):
    """
    Computes the regularization loss to ensure the matryoshka embeddings do not deviate
    significantly from the original embeddings.
    
    Args:
        original_embeddings: A tensor of shape (batch_size, embedding_dim) representing the original embeddings.
        matryoshka_embeddings: A tensor of shape (batch_size, mat_embedding_dim) representing the matryoshka embeddings.
        alpha: A regularization coefficient that controls the weight of the regularization term.
        
    Returns:
        loss: A scalar tensor representing the regularization loss.
    """
    # Compute the difference between original and matryoshka embeddings
    diff = original_embeddings - matryoshka_embeddings
    
    # Compute the L2 norm of the difference
    loss = torch.norm(diff, p=2, dim=1)
    
    # Return the mean loss over the batch, scaled by alpha
    return alpha * loss.mean()


# Equation 4 in paper
def unsupervised_objective_fn_loss(original_embeddings, matryoshka_embeddings, 
                                   k=5, alpha=1.0, beta=1.0):
    """
    Computes the overall unsupervised objective function loss as a combination of top-k similarity loss,
    alpha-scaled pairwise similarity loss, and beta-scaled regularization loss.
    
    Args:
        original_embeddings: A tensor of shape (batch_size, embedding_dim) representing the original embeddings.
        matryoshka_embeddings: A tensor of shape (batch_size, mat_embedding_dim) representing the matryoshka embeddings.
        k: The number of top similar embeddings to consider for the top-k similarity loss.
        alpha: A scaling factor for the pairwise similarity loss.
        beta: A scaling factor for the regularization loss.
        
    Returns:
        total_loss: A scalar tensor representing the combined unsupervised objective function loss.
    """
    # Compute the individual loss components
    topk_loss = topk_similarity_loss(original_embeddings, matryoshka_embeddings, k)
    pairwise_loss = pairwise_similarity_loss(original_embeddings, matryoshka_embeddings)
    reg_loss = regularization_loss(original_embeddings, matryoshka_embeddings, beta)
    
    # Combine the losses with the given scaling factors
    total_loss = topk_loss + alpha * pairwise_loss + beta * reg_loss
    
    return total_loss

## Training of Adaptor

### Unsupervised Implementation

#### Prepare datasets

We will use BEIR's NFCorpus and stick to the same train-test split pairs as in the paper

In [26]:
from datasets import load_dataset
from torch.utils.data import DataLoader, random_split
import torch

# Load the dataset
corpus_ds = load_dataset("BeIR/nfcorpus", "corpus")

# Access the 'corpus' dataset
dataset = corpus_ds['corpus']

# Define the split sizes
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Define batch size
batch_size = 32  # Adjust this as needed

# Create DataLoader for train and test datasets
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Example of iterating through the dataloaders
for batch in train_dataloader:
    print(batch)
    break  # Just for demonstration, remove this in actual training

for batch in test_dataloader:
    print(batch)
    break  # Just for demonstration, remove this in actual testing

{'_id': ['MED-4563', 'MED-5341', 'MED-3108', 'MED-844', 'MED-5134', 'MED-869', 'MED-1524', 'MED-1309', 'MED-5357', 'MED-1317', 'MED-666', 'MED-1179', 'MED-1950', 'MED-1587', 'MED-4913', 'MED-3044'], 'title': ['Nasal saline irrigations for the symptoms of chronic rhinosinusitis.', 'Effects of a low-fat, high-fiber diet and exercise program on breast cancer risk factors in vivo and tumor cell growth and apoptosis in vitro.', 'Epithelial barrier biology: good fences make good neighbours', 'Antifungal agents vs. boric acid for treating chronic mycotic vulvovaginitis.', 'Complementary feeding: a commentary by the ESPGHAN Committee on Nutrition.', 'Yerba Mate (Ilex paraguariensis) consumption is associated with higher bone mineral density in postmenopausal women.', 'Can Ambient Scent Enhance the Nightlife Experience?', 'Oat prevents obesity and abdominal fat distribution, and improves liver function in humans.', 'Can rye intake decrease risk of human breast cancer?', 'Avenanthramides inhibit

In [24]:
import torch
from torch.utils.data import Dataset

# Custom PyTorch dataset class that can handle pairs of embeddings
class PairDataset(Dataset):
    def __init__(self, dataset, pairs):
        self.dataset = dataset
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        idx1, idx2 = self.pairs[idx]
        text1 = self.dataset[idx1]['text']
        text2 = self.dataset[idx2]['text']
        return text1, text2


In [ ]:
from torch.utils.data import DataLoader

# Create train and test datasets
train_dataset = PairDataset(dataset, train_pairs)
test_dataset = PairDataset(dataset, test_pairs)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


#### Prepare Embedding Model and Matryoshka-Adaptor

In [13]:
from sentence_transformers import SentenceTransformer

# Embedding Model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Matryoshka-Adaptor
input_dim = 384 # Embedding dimension is 384 for embedding model (d in paper)
hidden_dim = input_dim # Let hidden layer dimensino equal input dimension
output_dim = 128 # Desired embedding dimension for final output (m in paper)
mat_adaptor = MatryoshkaAdaptor(input_dim, hidden_dim, output_dim)

#### Train Matryoshka-Adaptor

In [15]:
import torch
from torch.optim import Adam

def train(model, mat_adaptor, train_loader, loss_fn, epochs=10, lr=1e-3):
    # Define an optimizer for the MatryoshkaAdaptor parameters
    optimizer = Adam(mat_adaptor.parameters(), lr=lr)
    
    # Set MatryoshkaAdaptor to training mode
    mat_adaptor.train()

    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            text1, text2 = batch  # Unpack the batch

            # Generate embeddings for both texts
            ce_i = model.encode(text1, convert_to_tensor=True)  # model embedding i
            ce_j = model.encode(text2, convert_to_tensor=True)  # model embedding j

            # Forward pass through the MatryoshkaAdaptor
            che_i = mat_adaptor(ce_i)
            che_j = mat_adaptor(ce_j)

            # Compute loss
            loss = loss_fn((ce_i, ce_j), (che_i, che_j))

            # Backpropagation
            optimizer.zero_grad()  # Clear previous gradients
            loss.backward()        # Compute gradients
            optimizer.step()        # Update weights

            total_loss += loss.item()
            
            break

        # Print average loss for the epoch
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

In [16]:
train(model, mat_adaptor, train_loader, unsupervised_objective_fn_loss, epochs=1, lr=1e-3)

NameError: name 'unsupervised_objective_fn_loss' is not defined

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

## BEIR Evaluation

### Unmodified Model Performance for all-MiniLM-L6-v2

> Add blockquote



In [ ]:
import mteb
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model_name = 'all-MiniLM-L6-v2_BASE'

# Define the BEIR tasks you want to evaluate on
tasks = mteb.get_tasks(tasks=["NFCorpus"])

# Evaluate the model on the benchmark
evaluation = mteb.MTEB(tasks=tasks)
results = evaluation.run(model, output_folder=f"results/{model_name}")
